### Recurrent Neural Network (LSTM)

This is a basic convolutional approach to sequence prediction using convolutions through TensorFlow!

We begin by important any relevant packages, modules, and the DataProcessor class (for our pre-processed data). Then we get our dataset ready.

In [1]:
import os, sys
sys.path.append(os.path.abspath('..'))  # add parent directory to sys.path
from data_cleanup import DataProcessor
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the windowing parameters
# Use 10 hours of history
INPUT_WINDOW = 10  
# Predict the next 1 hours
OUTPUT_WINDOW = 1 

# Initialize the class
processor = DataProcessor(input_steps=INPUT_WINDOW, output_steps=OUTPUT_WINDOW)

# Run the pipeline
(X_train, y_train), (X_val, y_val), (X_test, y_test) = processor.load_and_process_data()

# Check the final shapes
print("\n--- Final Data Shapes ---")
print(f"X_train shape: {X_train.shape}  | y_train shape: {y_train.shape}")
print(f"X_val shape:   {X_val.shape}     | y_val shape:   {y_val.shape}")
print(f"X_test shape:  {X_test.shape}    | y_test shape:  {y_test.shape}")


Step 1/5: Fetching, cleaning, and engineering features...


c:\Python310\lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)
c:\Users\15302\Desktop\ECS171\ECS171G13\data_cleanup.py:77: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


Step 2/5: Resampling data to hourly and setting 'Global_active_power' as target...
Step 3/5: Splitting data and applying scaler...
Step 4/5: Creating time-series windows...
Step 5/5: Data processing complete.

--- Final Data Shapes ---
X_train shape: (25917, 10, 8)  | y_train shape: (25917, 1)
X_val shape:   (3614, 10, 8)     | y_val shape:   (3614, 1)
X_test shape:  (5028, 10, 8)    | y_test shape:  (5028, 1)


c:\Users\15302\Desktop\ECS171\ECS171G13\data_cleanup.py:118: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_hourly = df_hourly.fillna(method='ffill')


Next, we want to build the model itself. Keras makes this very simple at a high-level, so tuning is also easy to do.

In [ ]:
# Initialize the "order" of layers to be inserted. Will be a simple sequence.
model = Sequential()

# Add the first layer, which is an LSTM layer. We will choose to have 128 neurons, which sets the dimensionality seen
# throughout the LSTM layer.   
model.add(LSTM(units=128, input_shape=(10, 8)))

# The second layer is the final layer. This is a Dense one so we get one final hour to predict.
model.add(Dense(1))

# Set some parameters for how model will be optimized and how loss is calculated.
model.compile(optimizer='adam', loss='mean_squared_error')

Now we want to fit the model, train it, and determine an error metric.

In [ ]:
# This helper watches val_loss and stops training automatically if the model stops improving.
early_stopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

print("Starting model training...")
history = model.fit(
    X_train, 
    y_train, 
    epochs=10,                      
    batch_size=32,                  
    validation_data=(X_val, y_val), 
    callbacks=[early_stopper],      
    verbose=1
)
print("Training complete.")

print("Making predictions on the test set...")
# This gives you scaled predictions (e.g., 0.34, 0.52, etc.)
predictions_scaled = model.predict(X_test)

# Un-scale  model's predictions
unscaled_predictions = processor.inverse_transform_predictions(predictions_scaled)

# Un-scale y_test so we can compare using the exact same helper function.
unscaled_y_test = processor.inverse_transform_predictions(y_test)


mse = mean_squared_error(unscaled_y_test, unscaled_predictions)
print(f'\n--- Model Evaluation ---')
print(f'Test Set MSE (Mean Squared Error): {mse:.4f}')

# This is the human-readable version 
rmse = np.sqrt(mse)
print(f'Test Set RMSE (Root Mean Squared Error): {rmse:.4f}')
print(f'(This means, on average, the model was off by approx. {rmse:.2f} kilowatts)')